# Grasp-and-Lift(GAL) EEG Detection
## Hand movements の分類
- 被験者数: **12**
- 各被験者ごとの試行のデータ系列数: **10**
- 各被験者のひとつのデータ系列内の試行回数: **約30**(試行回数は各系列データごとに異なる) 

training set: 各被験者の最初の8つの試行のデータ系列<br/>
test set: 第9,10番目の試行のデータ系列<br/>

### ラベル
各GALには、6つのイベントを検出するタスク 
(それぞれのイベントにおいて2値分類(ラベル0,1))
　
1. HandStart
1. FirstDigitTouch
1. BothStartLoadPhase
1. LiftOff
1. Replace
1. BothReleased

これらのイベントは常に同じ順序で発生する<br/>
training setには、各件名+シリーズの組み合わせごとに2つのファイル<br/>

### データ
* *_data.csvファイルには、rawの32チャネルEEG(Electroencephalography, 脳波)データ（サンプリングレート500Hz）
* *_events.csvファイルには、すべてのイベントのフレーム・ワイズ・ラベル(1の連続)が含まれる
 * 6つのラベル列は、対応するイベントが±150ms（±75フレーム）以内に発生したかどうかに応じて、ゼロまたは1のいずれか

## 目標
#### 理想: イベントの窓全体を完璧に予測

## 注意
#### 未来データは使用できない(予測する系列の平均などはとれない)

## μ律動
7～12Hzのアーチ状の連続した波で，中心・頭頂部に一側性または両側性に出現する．
開眼時には減衰しないが，手を握るなどの運動や感覚刺激により抑制される

In [2]:
from IPython.display import HTML
HTML(r'<iframe width="560" height="315" src="https://www.youtube.com/embed/y3_Izuop2gY" frameborder="0" allowfullscreen></iframe>')

# パッケージの準備

In [10]:
import numpy as np
import scipy
import scipy.signal as signal
from scipy import fftpack
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from glob import glob
import os

from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
%matplotlib inline

# LeaveOneGroupOut交差検定
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import train_test_split
# AUCスコア
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score


from joblib import Parallel, delayed

# keras
from keras.utils.np_utils import to_categorical
import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.optimizers import SGD

from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.utils import plot_model
from keras.callbacks import TensorBoard



# 関数の準備

In [5]:
### データの読み込み ###

def prepare_data_train(fname):
    """ 訓練データの読み込み """
    # EEGデータ読み込み
    data = pd.read_csv(fname)
    # fnameイベントファイルの名前に変換
    events_fname = fname.replace('_data','_events')
    # イベントデータの読み込み
    labels= pd.read_csv(events_fname)
    clean=data.drop(['id' ], axis=1)#id列を削除
    labels=labels.drop(['id' ], axis=1)#id列を削除
    return  clean,labels

def prepare_data_test(fname):
    """ テストデータの読み込み """
    # EEGデータの読み込み
    data = pd.read_csv(fname)
    return data

In [6]:
### 前処理 ###

def preprocess_median_filter(X, kernel):
    """ Median filter"""
    X_m = signal.medfilt(X, kernel_size=kernel)
    return X_m

def preprocess_fir_filter(X, fc):
    """ FIR filter """
    fs = 500
    nyq = fs / 2.0  # ナイキスト周波数

    # フィルタの設計
    # ナイキスト周波数が1になるように正規化
    fe = fc / nyq      # カットオフ周波数1
    numtaps = 15          # フィルタ係数（タップ）の数（要奇数）

    b = scipy.signal.firwin(numtaps, fe) # Low-pass

    # FIRフィルタをかける
    X_FIR = scipy.signal.lfilter(b, 1, X)
    return X_FIR

def cut_off(X, fs):
    """ FFT処理後に高周波を取り除く"""
    # fs: カットオフ周波数[Hz]
    # 時系列のサンプルデータ作成
    n = X.shape[0]                         # データ数
    dt = 0.002                       # サンプリング間隔
    f = 500                           # 周波数

    # FFT 処理と周波数スケールの作成
    X_f = fftpack.fft(X)/(n/2)
    freq = fftpack.fftfreq(n, dt)

    # フィルタ処理
    # ここではカットオフ周波数以上に対応するデータを 0 にしている                          
    X_f2 = np.copy(X_f)
    X_f2[(freq > fs)] = 0
    X_f2[(freq < 0)] = 0

    # 逆 FFT 処理
    # FFT によるフィルタ処理では虚数部が計算されることがあるため
    # real 関数が必要(普段は必要ない)
    X_prep = np.real(fftpack.ifft(X_f2)*n)
    
    return X_prep


def data_preprocess_train(X):
    scaler= StandardScaler()
    X_prep = scaler.fit_transform(X)
#     X_prep = preprocess_fir_filter(X_prep, 100.0)
#     X_prep = cut_off(X, 50.0)

    #ここで他のpreprocessingを追加
    return X_prep

def data_preprocess_test(X):
    scaler= StandardScaler()
    X_prep = scaler.transform(X)
    #ここで他のpreprocessingを追加
    return X_prep

フィルタ処理はどうしたらいいかわからない

In [7]:
# ダウンサンプリングてきななにか
subsample=100

## 各GALごとに, 6 イベント(ラベル列):

1. HandStart
1. FirstDigitTouch
1. BothStartLoadPhase
1. LiftOff
1. Replace
1. BothReleased

In [8]:
cols = ['HandStart','FirstDigitTouch',
        'BothStartLoadPhase','LiftOff',
        'Replace','BothReleased']

# 訓練過程 

# 交差検証法を用いたAUC(Area Under the Curve)による予測能の比較
- AUC: ROC曲線(Receiver Operatorating Characteristic curve、受信者動作特性曲線)の面積
- 混合行列を定量的に比較し，予測能を判断するもの

## DeepNet

In [ ]:
#######number of subjects###############
subjects = range(1,13)
series = range(1, 9)
pred_tot = []
y_tot = []
global_auc = []
###loop on subjects and 8 series for train data + 2 series for test data
for i, subject in enumerate(subjects):
    print('Subject %d' % (subject))
    y_raw= []
    raw = []
    sequence = []
    auc_tot = []
    ################ READ DATA ################################################
    for ser in series:
        fname =  'input/train/subj%d_series%d_data.csv' % (subject,ser)
        data,labels=prepare_data_train(fname)
        raw.append(data)
        y_raw.append(labels)
        sequence.extend([ser]*len(data))

    X = pd.concat(raw)
    y = pd.concat(y_raw)
    #transform in numpy array
    #transform train data in numpy array
    X = np.asarray(X.astype(float))
    y = np.asarray(y.astype(float))
    sequence = np.asarray(sequence)
#     print(sequence.shape, y.shape, X.shape)
    y_binary = to_categorical(y[:, 0])
    
    
    ########### Palameter ######################################################
    n_in = len(X[0])
    n_hiddens = [200, 200, 200, 200]
    n_out = len(y_binary[0])
    p_keep = 0.5
    activation = 'relu'

    # model = Sequential()
    # for i, input_dim in enumerate(([n_in] + n_hiddens)[:-1]):
    #     model.add(Dense(n_hiddens[i], input_dim=input_dim))
    #     model.add(Activation(activation))
    #     model.add(Dropout(p_keep))

    # model.add(Dense(n_out))
    # model.add(Activation('softmax'))

    # model.compile(loss='categorical_crossentropy',
    #               optimizer=SGD(lr=0.00001),
    #               metrics=['accuracy'])

    epochs = 1
    batch_size = 200

    ################ Train classifiers ########################################
    cv = LeaveOneGroupOut()
    cv.get_n_splits(groups=sequence)
    cvscores = []
    # pred = np.empty((X.shape[0],6))
    cvfold = 1
    auc_tot = []

    for train, test in cv.split(X, y_binary, sequence):
        print('\nFold: ', cvfold, '\n')
        cvfold = cvfold + 1
        X_train, X_test = X[train], X[test]
        y_train, y_test = y_binary[train], y_binary[test]
        #apply preprocessing
    #     X_train = data_preprocess_train(X_train)
    #     X_test = data_preprocess_test(X_test)

        model = Sequential()
        for i, input_dim in enumerate(([n_in] + n_hiddens)[:-1]):
            model.add(Dense(n_hiddens[i], input_dim=input_dim))
            model.add(BatchNormalization())
            model.add(Activation(activation))
            model.add(Dropout(p_keep))

        model.add(Dense(n_out))
        model.add(Activation('softmax'))

        model.compile(loss='categorical_crossentropy',
                      optimizer=SGD(lr=0.001),
                      metrics=['accuracy'])
        model.fit(X_train, y_train, epochs=epochs,
                     batch_size=batch_size, verbose=1)

        scores = model.evaluate(X_test, y_test, verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        cvscores.append(scores[1] * 100)
        pred = model.predict_proba(X_test)
        auc = roc_auc_score(y_test,pred)    
        auc_tot.append(auc)
        print(auc, '\n')

    auc_tot = np.asarray(auc_tot)
    print(auc_tot)
    print('Mean AUC: ', np.mean(auc_tot))
    global_auc.append(np.mean(auc_tot))

        
#     preds = Parallel(n_jobs=6)(delayed(predict)(clfs[i],X_test) for i in range(6))
#     pred[test,:] = np.concatenate(preds,axis=1)

Subject 1

Fold:  1 

Epoch 1/1
1302896/1302896 [==============================] - 108s - loss: 0.1536 - acc: 0.9698   
acc: 95.73%
119264/119496 [============================>.] - ETA: 0s
Fold:  2 

Epoch 1/1
1150438/1150438 [==============================] - 107s - loss: 0.1788 - acc: 0.9634   
acc: 98.12%
271680/271954 [============================>.] - ETA: 0s
Fold:  3 

Epoch 1/1
 685000/1204778 [================>.............] - ETA: 48s - loss: 0.1911 - acc: 0.9600

In [ ]:
global_auc
sum(global_auc) / float(len(global_auc))

In [1]:
fig, ax = plt.subplots()
x = range(1, len(global_acc)+1)
plt.bar(x, global_auc)
plt.xlabel('Subject')
plt.ylabel('AUC')
plt.title('CV auc for each subject')
plt.savefig('cross_val_auc_subject.png' ,bbox_inches='tight')

NameError: name 'np' is not defined

In [28]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

96.86% (+/- 1.18%)


In [ ]:
fig, ax = plt.subplots()
x = range(1, len(cvscores)+1)
plt.bar(x, cvscores)
plt.xlabel('Subject')
plt.ylabel('AUC')
plt.title('CV auc for each subject')
plt.savefig('cross_val_auc_subject.png' ,bbox_inches='tight')